<a href="https://colab.research.google.com/github/andypedre2016/Data-MIning-Class/blob/main/otto-group-classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To solve this challege I will a using GXBoost classifier.
Steps:
1. Import Data
2. Divide data into features and labels 
3. One Hot Encode the features 
4. Create a GXBoost classifier 
5. Train classifier
6. Make predictions 

In [ ]:
#Importing the csv files
#Once imported, using the read_csv command to read them 

import pandas as pd

salesTrain = pd.read_csv('train.csv')
salesTest = pd.read_csv('test.csv')

#set The 'id' column as the index
salesTrain = salesTrain.set_index('id')
salesTest = salesTest.set_index('id')

#Using head() method to take a look at the dataset 
salesTrain.head()

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
id,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,Class_1
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Class_1
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Class_1
4,1,0,0,1,6,1,5,0,0,1,...,0,1,2,0,0,0,0,0,0,Class_1
5,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,Class_1


In [ ]:
#Using head() method to take a look at the dataset 
salesTest.head()

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,...,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93
id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,3,...,0,0,11,1,20,0,0,0,0,0
2,2,2,14,16,0,0,0,0,0,0,...,0,0,0,0,0,4,0,0,2,0
3,0,1,12,1,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,1
4,0,0,0,1,0,0,0,0,0,0,...,0,3,1,0,0,0,0,0,0,0
5,1,0,0,1,0,0,1,2,0,3,...,0,0,0,0,0,0,0,9,0,0


In [ ]:
#taking a look at the shape of our data
print('test shape:', salesTest.shape)
print('train shape: ', salesTrain.shape)

test shape: (144368, 93)
train shape:  (61878, 94)


In [ ]:
#Now dividing the data into features and labels

#gettin rid of the 'target' column and setting the rest of the columns as the features
sales_features = salesTrain.copy()
sales_features = sales_features.drop('target', axis = 1) 

#getting the 'target' column and setting it as the labels
sales_labels = salesTrain['target']

#taking a quick look at the features
sales_labels

id
1        Class_1
2        Class_1
3        Class_1
4        Class_1
5        Class_1
          ...   
61874    Class_9
61875    Class_9
61876    Class_9
61877    Class_9
61878    Class_9
Name: target, Length: 61878, dtype: object

In [ ]:
#Because I am going to be using a GXBoost classifier
# I need to One Hot Encode my data so I will get better accuracy score 

#importing OneHotEncoder from the sklearn library
from sklearn.preprocessing import OneHotEncoder

#Initialazing OneHotEncoder
ohe = OneHotEncoder(handle_unknown= 'ignore')

#Fitting the OneHotEncoder with the features
salesOHE = ohe.fit_transform(sales_features)
salesOHE

<61878x3960 sparse matrix of type '<class 'numpy.float64'>'
	with 5754654 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.model_selection import train_test_split

#Now I will divide the features and the labels into training and test sets
#I'll be setting the test size to the 20% of the data and the left 80% to training 
sales_train_features, sales_test_features, sales_train_labels, sales_test_labels = train_test_split(salesOHE, sales_labels, test_size = 0.2)
sales_train_features

<49502x3960 sparse matrix of type '<class 'numpy.float64'>'
	with 4603686 stored elements in Compressed Sparse Row format>

In [ ]:
#taking a look at the format of the test features
sales_test_features.shape

(12376, 3960)

In [ ]:
from xgboost import XGBClassifier

#Creating a set of paramater which our model is goint to be instantiated from
params = { "n_estimators": 400, 'tree_method':'gpu_hist', 'predictor':'gpu_predictor', 'max_depth': 6  }

#Instantiateing XGBoost model
model = XGBClassifier(**params)
model

XGBClassifier(max_depth=6, n_estimators=400, predictor='gpu_predictor',
              tree_method='gpu_hist')

In [ ]:
#Using the train features and the train labels to train the Model 
model.fit(sales_train_features, sales_train_labels)

XGBClassifier(max_depth=6, n_estimators=400, objective='multi:softprob',
              predictor='gpu_predictor', tree_method='gpu_hist')

In [ ]:
from sklearn.metrics import accuracy_score

#Getting the accuracy of the model
#Using the test features to test how well or the model predicts
sales_predictions = model.predict(sales_test_features) #test features

#Getting the accuracy of the model, comparing how well the model predicts the labels
accuracy_score(sales_test_labels, sales_predictions) #test labels

0.7904007756948933

## Trying to get a better accuracy

To be able to obtain a better accuracy I will be playing with the hyperparameters 

In [ ]:
from math import hypot
from sklearn.model_selection import GridSearchCV

#I want to find the best hyperparameters 
#Create a grid with the possible options
param_grid = [
    {'max_depth' : [2, 4, 6, 8, 10],
     'n_estimators' : [50, 100, 150, 200, 400]}
]


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

#Finding the best combination of hyperparameters
#I want the search algorithm to select 3 combinations of hyperparameters param_comb at random.
param_comb = 3
folds=3
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)
random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=param_comb,  n_jobs=-1, 
                                   cv=skf.split(sales_train_features,sales_train_labels), verbose=3)

In [ ]:
#Looking for a combination of hyperparameters
%%time 
grid_result = random_search.fit(sales_train_features, sales_train_labels)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


CPU times: user 5min 49s, sys: 41 s, total: 6min 30s
Wall time: 26min 53s


In [ ]:
#Taking a look at the combination of hyperparameters gotten
random_search.best_params_

{'n_estimators': 200, 'max_depth': 10}

In [ ]:
#Making preductiions with our GXBoost model train and fitted with the combiantion of hyperparameters I got from the lines above
predictions = random_search.best_estimator_.predict(sales_test_features)

In [ ]:
#Calculation the accuracy score
#How well the model is able to preduct on the products 
accuracy_score(sales_test_labels, predictions)

0.7968648998060762